In [1]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
from dotenv import load_dotenv
import os
load_dotenv()
os.getenv("GOOGLE_APPLICATION_CREDENTIALS"), os.getcwd()

('config/genf-446213-da3d943bbef2.json',
 '/Users/sigvardbratlie/Documents/Projects/genf/playground')

In [2]:
client = bigquery.Client()

### All Official users

In [6]:
m = pd.read_excel("/Users/sigvardbratlie/Downloads/m.xlsx", engine="openpyxl")
m.columns = [x.strip().lower().replace(" ","_") for x in m.columns]
m = m.loc[:,["person_id", "first_name", "last_name","display_name" ,"birthdate", "gender", "email",
             "cellphone", "address_line_1","city","postal_code", "mother_name","father_name"]]
m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   person_id       166 non-null    int64         
 1   first_name      166 non-null    object        
 2   last_name       166 non-null    object        
 3   display_name    166 non-null    object        
 4   birthdate       166 non-null    datetime64[ns]
 5   gender          166 non-null    object        
 6   email           161 non-null    object        
 7   cellphone       161 non-null    object        
 8   address_line_1  166 non-null    object        
 9   city            166 non-null    object        
 10  postal_code     166 non-null    int64         
 11  mother_name     166 non-null    object        
 12  father_name     165 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 17.0+ KB


### Fetch all registrations for active user emails

In [19]:
query_reg = "SELECT * FROM registrations.sesong_25_26"
df25 = client.query(query_reg).to_dataframe()
query_reg = "SELECT * FROM registrations.sesong_24_25"
df24 = client.query(query_reg).to_dataframe()

E0000 00:00:1766412811.994161  516864 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1766412814.216438  516864 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [18]:
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   dato            1006 non-null   datetime64[us, UTC]
 1   navn            1006 non-null   object             
 2   alder           1006 non-null   Int64              
 3   rolle           1006 non-null   object             
 4   kommentar       1006 non-null   object             
 5   gruppe          1006 non-null   object             
 6   prosjekt        1006 non-null   object             
 7   timer           1006 non-null   float64            
 8   kontonr         1006 non-null   Int64              
 9   epost           1006 non-null   object             
 10  antall_enheter  1006 non-null   Int64              
 11  kostnad         1006 non-null   float64            
dtypes: Int64(3), datetime64[us, UTC](1), float64(2), object(6)
memory usage: 97.4+ KB


In [ ]:
#rename and drop duplicates
df_r = df_reg.loc[:,["navn","kontonr","rolle", "epost"]].drop_duplicates()

rename_map = {"navn" : "display_name",
              "epost" : "email",
              }
df_r.rename(columns=rename_map, inplace=True)

In [181]:
#normalize names and emails
df_r["display_name"] = df_r["display_name"].str.strip().str.lower()
df_r["first_name"] = df_r["display_name"].apply(lambda x: x.strip().lower() if len(x.strip().split(" ")) == 1 else " ".join(x.strip().split(" ")[0:-1]).lower())
df_r["last_name"] = df_r["display_name"].apply(lambda x: "" if len(x.strip().split(" ")) == 1 else x.strip().split(" ")[-1].lower())
df_r["email"] = df_r["email"].str.lower().str.strip()

#custom corrections
df_r.loc[df_r["email"] == "leonpfeifer61@gmail.com", "email"] = "leon.pfeifer.m@gmail.com"
df_r.loc[df_r["email"] == "calvin.noa.marques@icloud.con", "email"] = "calvin.noa.marques@icloud.com"
df_r.loc[df_r["kontonr"] == "5396542538", "kontonr"] = 12256099088

#rm corrected duplicates
df_r.drop_duplicates(subset=["display_name","email"], inplace=True)

## Fetch all official members from BCC 2007-2012

In [139]:
all = []
for k in ["07", "08", "09", "10", "11", "12"]:
    query_members = f"SELECT * FROM members.20{k}"
    df_members = client.query(query_members).to_dataframe()
    all.append(df_members)
df_m = pd.concat(all).reset_index(drop=True)

E0000 00:00:1766311054.642249   27917 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1766311057.025677   27917 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1766311059.228382   27917 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1766311060.926189   27917 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1766311062.660221   27917 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1766311064.315483   27917 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [141]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   person_id        163 non-null    Int64         
 1   active_status    75 non-null     object        
 2   first_name       163 non-null    object        
 3   last_name        163 non-null    object        
 4   display_name     163 non-null    object        
 5   birthdate        163 non-null    datetime64[us]
 6   age              163 non-null    Int64         
 7   gender           163 non-null    object        
 8   church_name      163 non-null    object        
 9   membership_type  75 non-null     object        
 10  cellphone        157 non-null    object        
 11  address_line_1   139 non-null    object        
 12  city             75 non-null     object        
 13  postal_code      75 non-null     Int64         
 14  mother_name      99 non-null     object   

In [147]:
dfm = df_m.loc[:,["person_id","first_name","last_name","display_name","birthdate","cellphone",]]
dfm["rolle"] = dfm["birthdate"].apply(lambda x : "genf" if x.year in [2012,2011,2010] else "hjelpementor" if x.year in [2009,2008] else "mentor")
dfm["first_name"] = dfm["first_name"].str.lower().str.strip()
dfm["last_name"] = dfm["last_name"].str.lower().str.strip()
dfm["display_name"] = dfm["display_name"].str.lower().str.strip()

In [148]:
dfm

,person_id,first_name,last_name,display_name,birthdate,cellphone,rolle
0,42403,othilie smith,nielsen,othilie smith nielsen,2007-04-05,+47 47 62 60 15,mentor
1,41647,michelle,pfeifer,michelle helen pfeifer,2007-04-23,+47 472 40 800,mentor
2,41629,eunike,skare,eunike skare,2007-03-14,+47 483 06 257,mentor
3,41617,ida lin,baumann,ida lin baumann,2007-04-19,+47 969 18 919,mentor
4,42250,savannah,schmidt,savannah schmidt,2007-11-25,+47 48 40 85 23,mentor
...,...,...,...,...,...,...,...
158,46873,gilbert,haukelidsæter,gilbert haukelidsæter,2012-04-28,+47 979 62 804,genf
159,46983,charlie,irving,charlie irving,2012-07-29,+47 97 32 67 69,genf
160,46496,calvin noa,marques,calvin noa marques,2012-01-30,+47 973 12 922,genf
161,46628,rasmus,nevanlinna,rasmus nevanlinna,2012-03-04,+47 911 97 740,genf


In [149]:
#merge by display_name
m_dn = pd.merge(df_r, dfm[["display_name", "person_id", "birthdate","cellphone"]], how="inner", on=["display_name",])

In [182]:
#merge by first_name & rolle
m_fn = pd.merge(df_r, dfm[["first_name", "rolle", "display_name","person_id", "birthdate","cellphone"]], how="inner", on=["first_name","rolle"])
m_fn = m_fn.loc[(~m_fn["email"].isin(m_dn["email"])) & (~m_fn["person_id"].isin(m_dn["person_id"])) , :]

In [183]:
m_fn.loc[m_fn["person_id"] == 41647, : ] = pd.NA #fjerner michelle pfeifer feil match
m_fn = m_fn.dropna()
m_fn.rename(columns={"display_name_y" : "display_name"}, inplace=True)
m_fn.drop(columns=["display_name_x"], inplace=True)

In [186]:
df = pd.concat([m_dn,m_fn])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, 0 to 70
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   display_name  67 non-null     object        
 1   kontonr       67 non-null     Int64         
 2   rolle         67 non-null     object        
 3   email         67 non-null     object        
 4   first_name    67 non-null     object        
 5   last_name     67 non-null     object        
 6   person_id     67 non-null     Int64         
 7   birthdate     67 non-null     datetime64[us]
 8   cellphone     65 non-null     object        
dtypes: Int64(2), datetime64[us](1), object(6)
memory usage: 5.4+ KB


In [185]:
df.sample(10)

,display_name,kontonr,rolle,email,first_name,last_name,person_id,birthdate,cellphone
56,verena myhre,0,genf,myhreverena@gmail.com,verena,,46333,2011-10-24,+47 484 04 155
15,jonathan schmidt,37901886076,hjelpementor,schmidtygutt@gmail.com,jonathan,,44066,2009-07-24,+47 484 04 230
0,leon pfeifer,60312861634,genf,leon.pfeifer.m@gmail.com,leon,pfeifer,44406,2010-03-06,+47 41 62 52 72
21,elliot haukelidsæter,21401153052,genf,elliothaukeli@outlook.com,elliot,,44338,2010-02-14,+47 476 50 454
13,leon neumüller,61851101313,hjelpementor,leon@neumuller.net,leon,,43650,2009-04-09,+47 907 10 408
3,collin bratlie,62060850599,hjelpementor,collin.bratlie@gmail.com,collin,bratlie,42457,2008-05-02,+47 984 98 462
24,melina bratlie,62061375649,genf,melina.bratlie@icloud.com,melina,,44499,2010-05-05,+47 41 15 07 80
14,linus haukelidsæter,11004593229,genf,linushaukelidsaeter2@gmail.com,linus,haukelidsæter,44683,2010-07-24,+47 917 70 966
22,deborah schmidt,37901933961,genf,debsch11@icloud.com,deborah,schmidt,45640,2011-02-05,None
7,kevin hud,12253454014,hjelpementor,kevin.hud0505@gmail.com,kevin,hud,43602,2009-05-05,+47 407 61 042


In [163]:
df_m.loc[df_m["email"].notna(), :]

,person_id,active_status,first_name,last_name,display_name,birthdate,age,gender,church_name,membership_type,cellphone,address_line_1,city,postal_code,mother_name,father_name,email,address_line_4
75,45511,NaN,Amarins Evita,Boek,Amarins Evita Boek,2010-12-22,13,Female,Oslo og Follo,NaN,+47 955 25 157,Kroerveien 640,NaN,<NA>,NaN,NaN,amarinsboek@outlook.com,1435 ÅS
76,45440,NaN,Edith,Haukelidsæter,Oline Haukelidsæter,2010-10-27,13,Female,Oslo og Follo,NaN,+47 920 70 587,Bjastadveien 31,NaN,<NA>,NaN,NaN,olinehauk@gmail.com,1408 KRÅKSTAD
77,45338,NaN,Sofie,Haukelidsæter,Sofie Haukelidsæter,2010-10-05,13,Female,Oslo og Follo,NaN,+47 940 84 031,Marie Kirkerudsv. 15,NaN,<NA>,NaN,NaN,sofiehauk10@gmail.com,1407 VINTERBRO
78,45084,NaN,Livia,Kristiansen,Livia Kristiansen,2010-09-16,13,Female,Oslo og Follo,NaN,+47 94 06 94 58,Løkenveien 11,NaN,<NA>,NaN,NaN,livia@kristiansen.tv,1408 Kråkstad
79,46324,NaN,Olivia,Nevanlinna,Olivia Nevanlinna,2010-10-08,13,Female,Oslo og Follo,NaN,+47 466 26 648,Kråkstadveien 105 b,NaN,<NA>,NaN,NaN,olivianevanlinna@gmail.com,1408 KRÅKSTAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,46727,NaN,William Samuel,Christensen,William Samuel Christensen,2011-05-20,13,Male,Oslo og Follo,NaN,+47 930 00 157,Tomterveien 42,NaN,<NA>,NaN,NaN,williamchristensen2005@gmail.com,1408 KRÅKSTAD
135,46104,NaN,Kian-Andre,Debelak,Kian-Andre Debelak,2011-05-04,13,Male,Oslo og Follo,NaN,+47 923 21 576,Kråkstadveien 114 B,NaN,<NA>,NaN,NaN,kian@debelak.org,1408 KRÅKSTAD
136,45839,NaN,Oliver,Gangsø,Oliver Gangsø,2011-04-30,13,Male,Oslo og Follo,NaN,+47 413 00 427,Gamle Kongevei 14 B,NaN,<NA>,NaN,NaN,oliver.gangso@gmail.com,1423 SKI
137,45618,NaN,Mikael,Hud,Mikael Hud,2011-01-11,13,Male,Oslo og Follo,NaN,+47 973 38 811,Fålesloråsen 57,NaN,<NA>,NaN,NaN,mikaelhud11@gmail.com,1407 VINTERBRO


In [167]:
m_e = pd.merge(df_r, df_m[["email", "display_name", "person_id", "birthdate","cellphone"]], how="inner", on=["email",])
m_e = m_e.loc[(~m_e["person_id"].isin(df["person_id"])) , :]
m_e.rename(columns={"display_name_y" : "display_name"}, inplace=True)
m_e.drop(columns=["display_name_x"], inplace=True)
m_e

,kontonr,rolle,email,first_name,last_name,display_name,person_id,birthdate,cellphone
1,21401111716,genf,jennycb8765@gmail.com,jenny,,Jenny Cornelia Baumann,44265,2010-02-02,+47 973 18 152
4,11004781343,genf,lucasekholt@icloud.com,lucas,ekholt,Lucas Michael Ekholt,44354,2010-02-24,+47 484 78 041
6,12268644359,genf,nagytristanm@gmail.com,tristan miguel,nagy,Tristan-Miguel Nagy,45641,2010-03-18,+47 99 67 18 00
7,81201369207,genf,enriqueleon2010@icloud.com,enrique,marques,Enrique Léon Marques,45368,2010-04-10,+47 922 23 299
11,91005905589,genf,scott.nevanlinna@gmail.com,scott,,Scott Andrew Nevanlinna,45128,2010-07-10,+47 980 95 965
15,62061545080,genf,dorschyx10@gmail.com,dorthe,schytt,Dorthe Isabel Schytt,45101,2010-09-14,+47 465 34 954
20,66341024618,genf,epicnicolay@gmail.com,kristian,,Kristian Anker Olstad,45342,2010-10-06,+47 948 58 003
23,61851100570,genf,ingridalvilde@icloud.com,alvilde,olstad,Ingrid Alvilde Olstad,45499,2010-10-27,+47 94 08 79 68
24,12272691728,genf,mariell.skare@gmail.com,mariell,skare,Belinda Mariell Skare,45449,2010-11-15,+47 484 45 229
30,40352266495,genf,elena.rose.frivold@gmail.com,elena,frivold,Elena Rose Hines Frivold,45794,2011-02-14,+47 92 21 04 70


In [168]:
df = pd.concat([df, m_e]).reset_index(drop=True)

In [ ]:
l = df_r.loc[(~df_r["email"].str.strip().str.lower().isin(df["email"])),:]
person_id_map = {"phebelauri@icloud.com" :46914,
                  "nilsennatanael@gmail.com" : 43242,
                  "laurien696@gmail.com" :43241 ,
                  "sienna.lovise@gmail.com" : 43603,
                  "f.amalie.g@outlook.com" :43857,
                    "michael.a.christensen@outlook.com":44357,
                    "casperdekort10@gmail.com" : 44236,
                    "elenifernandakris@icloud.com" : 44635,
                    "leandernooit@outlook.com" : 0,
                    "rubikscube540@gmail.com" : 45023,
                    "silvanschmi11@gmail.com" : 0,
                    "scottnooit@outlook.com" : 0,
                    "scott@bdbrodie.com" : 46483,
                    "calvin.noa.marques@icloud.com" : 46496,
                    "amiliesavannah12@gmail.com" : 46667,
                    "mirellastadven@gmail.com" : 46689,
                    "emiliosakrubjorgengen@gmail.com" : 46723,
                    "leahekholt@icloud.com" : 46898,
                    "aronmtwilley@gmail.com" : 47067,
                    "riannemercedes@gmail.com" : 47077,
                    "william.james.frivold@gmail.com" : 44104,
                    }

for k,v in person_id_map.items():
    l.loc[l["email"].str.strip().str.lower() == k.lower().strip(), "person_id"] = v

l_im = l.loc[(l["rolle"].str.strip().str.lower() != "mentor")]
m_l = pd.merge(l_im, df_m[["person_id","birthdate","cellphone"]], how="inner", on=["person_id",])
df = pd.concat([df, m_l]).reset_index(drop=True)

#
df_last = df_r.loc[(~df_r["email"].str.strip().isin(df["email"].str.strip())) & (df_r["rolle"].str.strip().str.lower() != "mentor") ]
map_person_id = {"jennycb8765@gmail.com" : 44265,
                 "lucasekholt@icloud.com" : 44354,
                 "nagytristanm@gmail.com" : 45641,
                 "enriqueleon2010@icloud.com" : 45368,
                 "scott.nevanlinna@gmail.com" : 45128,
                 "dorschyx10@gmail.com" : 45101,
                 "epicnicolay@gmail.com" : 45342,
                 "ingridalvilde@icloud.com" : 45499,
                 "mariell.skare@gmail.com" : 45449,
                 "elena.rose.frivold@gmail.com" : 45794,
                 "kian@debelak.org" : 46104,
                 "williamchristensen2005@gmail.com" : 46727,
                 "nelly.schumacher@outlook.com" : 46026,
                 "schmidtmarita81@gmail.com" : 46054,
                 "paulavhauk@gmail.com" : 49425,
                 "miasofienilsen@outlook.com" : 46350,
                 "sahnorway@gmail.com" : 46353,
                 "isabelle.simone.brinks@gmail.com" : 46430,
                 "klym2211@gmail.com" : 46779,}

for k,v in map_person_id.items():
    df_last.loc[df_last["email"].str.strip().str.lower() == k.lower().strip(), "person_id"] = v

In [230]:
df.to_csv("user_export_2007-2012.csv")

In [ ]:
job = client.load_table_from_dataframe(df[["person_id","email","kontonr"]],"members.specs")
job.result()

In [231]:
l = df_r.loc[(~df_r["email"].str.strip().str.lower().isin(df["email"])),:]
l_m = l.loc[(l["rolle"].str.strip().str.lower() == "mentor")]

In [232]:
l_m.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 0 to 160
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   display_name  40 non-null     object
 1   kontonr       40 non-null     Int64 
 2   rolle         40 non-null     object
 3   email         40 non-null     object
 4   first_name    40 non-null     object
 5   last_name     40 non-null     object
dtypes: Int64(1), object(5)
memory usage: 2.2+ KB


In [235]:
l_m.loc[l_m["last_name"] == "", :]

,display_name,kontonr,rolle,email,first_name,last_name
66,tim,12065804768,mentor,tim_baltz@hotmail.no,tim,
85,mathias,0,mentor,mathiashaukeli@gmail.com,mathias,
129,zeta,12083176405,mentor,zeta.elianna03@icloud.com,zeta,
158,othilie,0,mentor,othilienielsen@gmail.com,othilie,


In [236]:
map_lastname = {"tim_baltz@hotmail.no" : "Baltzersen",
                "mathiashaukeli@gmail.com" : "Haukelidsæter",
                "zeta.elianna03@icloud.com" : "Bjørgengen",
                "othilienielsen@gmail.com" : "Nielsen"}

for k,v in map_lastname.items():
    l_m.loc[l_m["email"].str.strip().str.lower() == k.lower().strip(), "last_name"] = v.lower().strip()

In [238]:
l_m.to_csv("mentor_export.csv")

In [240]:
df.drop_duplicates(subset=["person_id"], inplace=True)

In [241]:
df

,display_name,kontonr,rolle,email,first_name,last_name,person_id,birthdate,cellphone,birthdate_x,cellphone_x,birthdate_y,cellphone_y
0,leon pfeifer,60312861634,genf,leon.pfeifer.m@gmail.com,leon,pfeifer,44406.0,2010-03-06,+47 41 62 52 72,NaT,NaN,NaT,NaN
1,trond sigurd pfeifer,18131360793,mentor,pfeifer.tsp@gmail.com,trond sigurd,pfeifer,43092.0,2008-08-30,+47 465 21 152,NaT,NaN,NaT,NaN
2,lasse kristiansen,0,hjelpementor,laskri08@icloud.com,lasse,kristiansen,42398.0,2008-04-09,+47 940 73 138,NaT,NaN,NaT,NaN
3,collin bratlie,62060850599,hjelpementor,collin.bratlie@gmail.com,collin,bratlie,42457.0,2008-05-02,+47 984 98 462,NaT,NaN,NaT,NaN
4,annicka schytt,62061545027,hjelpementor,annschyx8@gmail.com,annicka,schytt,43519.0,2008-12-09,+47 465 17 709,NaT,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,paula vanessa,12279042369,genf,paulavhauk@gmail.com,paula,vanessa,49425.0,NaT,NaN,2011-09-18,+47 46 88 02 61,2011-09-18,+47 46 88 02 61
100,mia sofie nilsen,0,genf,miasofienilsen@outlook.com,mia sofie,nilsen,46350.0,NaT,NaN,2011-10-29,+47 966 48 304,2011-10-29,+47 966 48 304
101,samuel holt,0,genf,sahnorway@gmail.com,samuel,holt,46353.0,NaT,NaN,2011-11-03,+47 405 54 697,2011-11-03,+47 405 54 697
102,isabelle brinks,0,genf,isabelle.simone.brinks@gmail.com,isabelle,brinks,46430.0,NaT,NaN,2011-11-22,+47 41 59 39 48,2011-11-22,+47 41 59 39 48


In [255]:
df_bq = client.query("SELECT * FROM members.specs").to_dataframe()

E0000 00:00:1766316125.925932   27917 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [257]:
df_bq

,person_id,email,kontonr
0,NaN,leandernooit@outlook.com,60111733131
1,NaN,silvanschmi11@gmail.com,12286688754
2,NaN,scottnooit@outlook.com,61851101240
3,44265.0,jennycb8765@gmail.com,21401111716
4,44354.0,lucasekholt@icloud.com,11004781343
...,...,...,...
101,45084.0,livia@kristiansen.tv,98024579891
102,47193.0,theodor.schmidt2012@gmail.com,98200624122
103,41749.0,theokrist07@gmail.com,98520635270
104,45267.0,mdalen2010@gmail.com,8191008315081
